# Relax Bulk with CP2K
Use CP2K PBE to relax a certain structure

In [1]:
from examol.simulate.ase.utils import make_ephemeral_calculator
from examol.simulate.ase import ASESimulator
from examol.utils.conversions import write_to_string
from pathlib import Path
from ase.constraints import ExpCellFilter
from ase.optimize import BFGS
from ase.build import bulk

Configuration

In [2]:
atoms = bulk('Pd')
name = 'fcc-Pd'
out_dir = Path('relaxed-bulk')

## Make the Bulk Cell
Do whatever you need to start with a new cell

## Make the Simulator
The tool we'll use to make an ASE calculator

In [3]:
sim = ASESimulator(cp2k_command='/home/lward/Software/cp2k-2023.2/exe/local_cuda/cp2k_shell.ssmp')

## Run the optimization
We'll use ExaMol to make ASE calculators with our target settings.

In [4]:
xyz = write_to_string(atoms, 'extxyz')

In [5]:
with make_ephemeral_calculator(sim.create_configuration('cp2k_pbe_dzvp', xyz, 0, None)) as calc:

    # Start by computing the stresses acting on the cell
    calc.directory = 'run'
    atoms.calc = calc
    stresses = atoms.get_stress()
    print(f'Initial volume: {atoms.get_volume()}. Stress: {stresses[:3].sum() / 3:.2f}')
    
    # Run the optimization
    opt_atoms = ExpCellFilter(atoms)  # Allow ASE to optimize the lattice parameter
    opt = BFGS(opt_atoms)
    opt.run(fmax=0.001)
    stresses = atoms.get_stress()
    print(f'Final volume: {atoms.get_volume()}. Stress: {stresses[:3].sum() / 3:.2f}')

Initial volume: 14.715967249999999. Stress: 0.05
      Step     Time          Energy          fmax
BFGS:    0 08:52:40    -3469.093567         0.781361
BFGS:    1 08:53:02    -3469.099891         0.423853
BFGS:    2 08:53:21    -3469.102291         0.013849
BFGS:    3 08:53:36    -3469.102293         0.000236
Final volume: 14.394562659381776. Stress: 0.00


Save the relaxed structure

In [6]:
out_dir.mkdir(exist_ok=True)

In [7]:
with out_dir.joinpath(f'{name}.extxyz').open('w') as fp:
    print(write_to_string(atoms, 'extxyz'), file=fp)